In [1]:
from testsavant.guard import InputGuard, OutputGuard
from testsavant.guard.input_scanners import Anonymize
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def dummy_llm(input):
    return input
long_article = open("../docs/article.txt", "r").read()
print(len(long_article))

ts_api = InputGuard(
    API_KEY=os.environ.get("TEST_SAVANT_API_KEY"),
    PROJECT_ID=os.environ.get("TEST_SAVANT_PROJECT_ID"),
    remote_addr=os.environ.get("TEST_SAVANT_REMOTE_ADDR")
)
scanner = Anonymize(tag="base", threshold=0.6, use_faker=False, redact=False, entity_types=['CREDIT_CARD', 'US_SSN_RE'])
ts_api.add_scanner(scanner)

7810


In [3]:
prompt = "Mr Don Smith is a software engineer. He works at a company called TechCorp and he is a colleague of Jane Doe. They are working on a project together. The project is about developing a new software application that will help people manage their tasks more efficiently. Mr Smith is very excited about this project and believes it will be a great success at google"
result = ts_api.scan(prompt)
if result.is_valid:
    print("Prompt is valid")
else:
    print("Prompt is invalid")

Prompt is valid


In [4]:
result.sanitized_prompt


'Mr Don Smith is a software engineer. He works at a company called TechCorp and he is a colleague of Jane Doe. They are working on a project together. The project is about developing a new software application that will help people manage their tasks more efficiently. Mr Smith is very excited about this project and believes it will be a great success at google'

In [5]:
len(prompt), len(result.sanitized_prompt)

(361, 361)

In [6]:
len(prompt.split()), len(result.sanitized_prompt.split())

(65, 65)

In [7]:
xprompt = prompt + " "  + long_article
result = ts_api.scan(xprompt)
len(xprompt.split()), len(result.sanitized_prompt.split())

(1460, 1460)

In [8]:
nprompt = "But Sister Eulalie suddenly arose. 'Thank you, 'father, but my brother and I prefer to remain alone with her. This is our last chance to see her, and we wish to be together, all three of us, as we--we--used to be when we were small and our poor mo--mother----'\n\nGrief and tears stopped her; she could not continue.\n\nOnce more serene, the priest bowed, thinking of his bed. 'As you wish, my children.' He kneeled, crossed himself, prayed, arose and went out quietly, murmuring: 'She was a saint!'\n\nThey remained alone, the dead woman and her children. The ticking of the clock, hidden in the shadow, could be heard distinctly, and through the open window drifted in the sweet smell of hay and of woods, together with the soft moonlight. No other noise could be heard over the land except the occasional croaking of the frog or the chirping of some belated insect. "
for i in range(100):
    result = ts_api.scan(nprompt)